In [ ]:
import sys
root_dir = '../../../'
sys.path.append(root_dir)

In [1]:
import pickle
from utils import *

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Load data

In [3]:
lorenz = pd.read_csv(root_dir + '/data/lorenz.csv', header=None).iloc[:, 0].values
X_train = lorenz[4_000: 24_000]

In [4]:
with open('../../01_base/results-base-2.pickle', 'rb') as f:
    results = pickle.load(f)

predicted = results['predicted']
true = results['true']
# predicted = results['predicted'][:, :50]
# true = results['true'][:, :50]

## Get patterns and motifs

In [5]:
patterns = get_patterns(
    pattern_size=3, 
    max_distance=10, 
    patterns_percent=100
)

motifs = get_motifs(X_train, patterns)

print(len(motifs))
print(motifs[patterns[0]].shape)

1000
(19984, 4)


## Algorithm

In [6]:
with open('../02_train_classifiers/knn.pickle', 'rb') as f:
    classifier = pickle.load(f)

In [7]:
inputs = predicted

def task(Y_pred):
    counter = [-1, np.sum(np.isnan(Y_pred))]
    for i in range(10):
        if counter[-1] == counter[-2]:
            break
        Y_pred = get_new_predictions(
            X_train = X_train,
            old_predictions = Y_pred,
            motifs = motifs,
            distance_eps = 0.005,
            cluster_eps = 0.01, 
            min_samples = 5, 
            cluster_1_proportion_threshold = -1, 
            clusters_diff_threshold = -1, 
            sigma = 0,
            keep_previous_iter_predictions = True,
            last = False,
            possible_predictions_min_size = 80,
            return_possible_predictions = False,
            classifier = classifier,
            classifier_prob = False
        )
        counter.append(sum(np.isnan(Y_pred)))
    return Y_pred

results = run_concurrently(task, inputs)
predicted_new = np.stack(results, axis=0)

In [8]:
with open('results-new-knn-1.pickle', 'wb') as f:
    pickle.dump(predicted_new, f)